In [2]:
import numpy as np 
import cv2 as cv 
from matplotlib import pyplot as plt 

img = cv.imread('fisherman.jpg') #load image 
img_gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY) 

In [3]:
height, width = img_gray.shape 
ori_img_gray = np.zeros((height, width), np.float32) # gradient orientation of img_gray